In [1]:
import pandas as pd
import numpy as np
import psycopg2
import random
from random import randint
from configparser import ConfigParser

In [2]:
def config(filename='testdb.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [3]:
params = config()
def connect():
    conn = None
    try:
        # read connection parameters

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
        
        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
        # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.2, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


# 2. Создание таблиц:

In [4]:
def create_tables():
    commands = (
        """
        CREATE TABLE IF NOT EXISTS client (
            client_id SERIAL PRIMARY KEY,
            client_name VARCHAR(100) NOT NULL,
            age INT2 NOT NULL,
            client_phone VARCHAR(20) NOT NULL,
            address VARCHAR(100) NOT NULL,
            passport VARCHAR(20) NOT NULL,
            personal_rate FLOAT4 NOT NULL
        )
        
        """,
        """  
        
        CREATE TABLE IF NOT EXISTS agent (
            agent_id SERIAL PRIMARY KEY,
            agent_name VARCHAR(50) NOT NULL,
            personnel_num INT2 NOT NULL
        )
        
        """,
        """
        
        CREATE TABLE IF NOT EXISTS tariff (
            tariff_id SERIAL PRIMARY KEY,
            category VARCHAR(20) NOT NULL,
            tariff_name VARCHAR(100) NOT NULL,
            tariff_price INT2 NOT NULL,
            info VARCHAR(200) NOT NULL
        )
        
        """,
        """
        
        CREATE TABLE IF NOT EXISTS install (
            install_id SERIAL PRIMARY KEY,
            install_name VARCHAR(100) NOT NULL,
            install_price INT2 NOT NULL
        )
        
        """,
        """
        
        CREATE TABLE IF NOT EXISTS master (
            master_id SERIAL PRIMARY KEY,
            master_name VARCHAR(100) NOT NULL,
            master_phone VARCHAR(20) NOT NULL,
            region VARCHAR(10) NOT NULL
        )
        
        """,
        """
                
        CREATE TABLE IF NOT EXISTS agreement (
            agreement_id SERIAL PRIMARY KEY,
            date VARCHAR(15) NOT NULL,
            client_id INT2 NOT NULL,
            FOREIGN KEY (client_id) REFERENCES client (client_id) ON UPDATE CASCADE ON DELETE CASCADE,
            tariff_id INT2 NOT NULL,
            FOREIGN KEY (tariff_id) REFERENCES tariff (tariff_id) ON UPDATE CASCADE ON DELETE CASCADE,
            install_id INT2 NOT NULL,
            FOREIGN KEY (install_id) REFERENCES install (install_id) ON UPDATE CASCADE ON DELETE CASCADE,
            agent_id INT2,
            FOREIGN KEY (agent_id) REFERENCES agent (agent_id) ON UPDATE CASCADE ON DELETE CASCADE,
            master_id INT2,
            FOREIGN KEY (master_id) REFERENCES master (master_id) ON UPDATE CASCADE ON DELETE CASCADE,
            total_cost FLOAT NOT NULL
        )
        """)
    conn = None
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        # create table one by one
        for command in commands:
            cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()


if __name__ == '__main__':
    create_tables()
    
print("Таблицы успешно созданы в PostgreSQL")

Таблицы успешно созданы в PostgreSQL


### Функция получения заголовков таблицы:

In [9]:
from psycopg2 import sql, connect



# define a function that gets the column names from a PostgreSQL table
def get_columns_names(table):
    conn = None
    try:
        # declare a new PostgreSQL connection object
        conn = psycopg2.connect(**params)
    
    # declare an empty list for the column names
        column_names = []

    # declare cursor objects from the connection    
        col_cursor = conn.cursor()

        sql = '''SELECT * FROM {}'''.format(table)
        col_cursor.execute(sql)
        column_names = [desc[0] for desc in col_cursor.description]

        # close the cursor object to prevent memory leaks
        col_cursor.close()

    except Exception as err:
        print ("get_columns_names ERROR:", err)

    # return the list of column names
    return column_names
get_columns_names('client')

['client_id',
 'client_name',
 'age',
 'client_phone',
 'address',
 'passport',
 'personal_rate']

### Функция вывода таблицы PostgreSQL в Pandas:

In [5]:
def postgres_to_dataframe(select_query):
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    try:
        cur.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cur.close
        return 1
    
    column_names = [desc[0] for desc in cur.description]
    tupples = cur.fetchall()
    cur.close
    
    df = pd.DataFrame(tupples, columns = column_names)
    return df
    

### Функция заполнения таблиц путём INSERT INTO:

In [11]:
def insert(table_name, record_to_insert, x):
    try:
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        columns = get_columns_names(table_name)
        columns_str = ""
        value_str = ""
        sign = "%s"
        for i in range(len(columns)):
            if (i != len(columns) -1):
                columns_str+="{},".format(columns[i])
                value_str+="{},".format(sign)
            else:
                columns_str+=columns[i]
                value_str+=sign
        
        # execute the INSERT statement
        postgres_insert_query = f""" INSERT INTO {table_name} ({columns_str}) VALUES ({value_str})"""
        cur.execute(postgres_insert_query, record_to_insert)
        conn.commit()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        print(f"{x}. Запись успешно добавлена в таблицу {table_name}")

# 3. Заполнение таблиц данными через функцию insert:

In [109]:
addr = []
streets = ["улица Академика Варги", "улица Тушинская", "улица Орджоникидзе", "Кутузовский проспект", "Хованская улица", "улица Лобачиха", "Вековая улица", "улица 26-ти Бакинских Комиссаров", "Мирской переулок", "улица Гончарова", "улица Зорге", "Новозаводская улица"]
for i in range(0, len(streets)):
    addr.append('{0}, {1}, д. {2}, кв. {3}'.format("г. Москва", streets[i], randint(5, 131), randint(1, 120)))
names = ["Звягина Людмила Арсеньевна,", "Базин Дмитрий Георгиевич", "Бакшаева Арина Антоновна", "Мажов Филипп Денисович", "Азарова Елена Геннадьевна", "Бакаринцев Игнат Ефимович", "Бичурин Илья Анатольевич", "Шеповалов Яков Семенович", "Моряева Наталья Сергеевна", "Труш Арсений Аркадьевич", "Катасонова Клара Витальевна", "Гришаева Инна Леонтьевна"]
phones = ["+7(915)944-86-94", "+7(906)869-66-00", "+7(926)869-11-59", "+7(907)745-95-25", "+7(916)603-20-81", "+7(499)094-25-02", "+7(925)405-11-55", "+7(910)252-30-10", "+7(903)668-19-46", "+7(495)408-58-04", "+7(964)225-44-40", "+7(495)343-00-78"]
rates = [1, 0.75, 0.5, 0.25]

In [110]:
####Вставка данных в таблицы
for i in range(0, 12): 
    record_to_insert = (i+1, names[i], randint(19, 55), phones[i], addr[i], "45{0} {1}".format(randint(11,19), randint(444444, 777777)), random.choice(rates))
    insert('client', record_to_insert, i)

0. Запись успешно добавлена в таблицу client
1. Запись успешно добавлена в таблицу client
2. Запись успешно добавлена в таблицу client
3. Запись успешно добавлена в таблицу client
4. Запись успешно добавлена в таблицу client
5. Запись успешно добавлена в таблицу client
6. Запись успешно добавлена в таблицу client
7. Запись успешно добавлена в таблицу client
8. Запись успешно добавлена в таблицу client
9. Запись успешно добавлена в таблицу client
10. Запись успешно добавлена в таблицу client
11. Запись успешно добавлена в таблицу client


In [135]:
postgres_to_dataframe('''SELECT * FROM client''')

,client_id,client_name,age,client_phone,address,passport,personal_rate
0,1,"Звягина Людмила Арсеньевна,",49,+7(915)944-86-94,"г. Москва, улица Академика Варги, д. 92, кв. 88",4512 715814,0.50
1,2,Базин Дмитрий Георгиевич,35,+7(906)869-66-00,"г. Москва, улица Тушинская, д. 126, кв. 75",4514 632683,0.50
2,3,Бакшаева Арина Антоновна,55,+7(926)869-11-59,"г. Москва, улица Орджоникидзе, д. 73, кв. 114",4515 627734,0.25
3,4,Мажов Филипп Денисович,44,+7(907)745-95-25,"г. Москва, Кутузовский проспект, д. 94, кв. 75",4511 511038,0.25
4,5,Азарова Елена Геннадьевна,34,+7(916)603-20-81,"г. Москва, Хованская улица, д. 45, кв. 53",4517 611837,0.25
5,6,Бакаринцев Игнат Ефимович,39,+7(499)094-25-02,"г. Москва, улица Лобачиха, д. 43, кв. 19",4516 538217,1.00
6,7,Бичурин Илья Анатольевич,48,+7(925)405-11-55,"г. Москва, Вековая улица, д. 127, кв. 86",4512 681346,0.75
7,8,Шеповалов Яков Семенович,40,+7(910)252-30-10,"г. Москва, улица 26-ти Бакинских Комиссаров, д...",4515 639699,1.00
8,9,Моряева Наталья Сергеевна,22,+7(903)668-19-46,"г. Москва, Мирской переулок, д. 74, кв. 75",4511 760845,0.75
9,10,Труш Арсений Аркадьевич,30,+7(495)408-58-04,"г. Москва, улица Гончарова, д. 123, кв. 22",4512 500050,1.00


In [113]:
names = ["Еремин Тимофей Адрианович", "Смирнова Виктория Константиновна", "Устинова Вероника Михайловна", "Моисеев Александр Ильич", "Баранов Иван Иванович", "Устинова Юлия Кирилловна", "Ерофеев Богдан Ярославович", "Осипова Эмилия Павловна"]
nums = []
for j in range(22, 88):
    nums.append(j)
nums = random.sample(nums, 8)
for i in range(0, 8):
    record_to_insert = (i+1, names[i], nums[i])
    insert('agent', record_to_insert, i)

0. Запись успешно добавлена в таблицу agent
1. Запись успешно добавлена в таблицу agent
2. Запись успешно добавлена в таблицу agent
3. Запись успешно добавлена в таблицу agent
4. Запись успешно добавлена в таблицу agent
5. Запись успешно добавлена в таблицу agent
6. Запись успешно добавлена в таблицу agent
7. Запись успешно добавлена в таблицу agent


In [136]:
postgres_to_dataframe('''SELECT * FROM agent''')

,agent_id,agent_name,personnel_num
0,1,Еремин Тимофей Адрианович,81
1,2,Смирнова Виктория Константиновна,66
2,3,Устинова Вероника Михайловна,65
3,4,Моисеев Александр Ильич,82
4,5,Баранов Иван Иванович,84
5,6,Устинова Юлия Кирилловна,48
6,7,Ерофеев Богдан Ярославович,33
7,8,Осипова Эмилия Павловна,29


In [117]:
names = ["Демьянов Кирилл Степанович", "Морозов Артур Тимурович", "Ефимов Сергей Богданович", "Ушаков Артём Германович", "Исаков Владимир Александрович", "Одинцов Владислав Романович", "Андреев Илья Иванович", "Черных Вадим Владимирович", "Демина Полина Валерьевна", "Сидоров Платон Владимирович"]
phones = ["+7(909)291-72-77", "+7(999)677-11-60", "+7(916)296-55-85", "+7(916)001-53-82", "+7(925)010-47-87", "+7(903)331-09-85", "+7(906)073-38-27", "+7(499)755-73-93", "+7(926)255-89-09", "+7(915)019-98-76"]
regions = ["ЦАО", "САО", "СВАО", "ВАО", "ЮВАО", "ЮАО", "ЮЗАО", "ЗАО", "СЗАО", "ЗелАО"]
for i in range(0, 10):
    record_to_insert = (i+1, names[i], phones[i], random.choice(regions))
    insert('master', record_to_insert, i)

0. Запись успешно добавлена в таблицу master
1. Запись успешно добавлена в таблицу master
2. Запись успешно добавлена в таблицу master
3. Запись успешно добавлена в таблицу master
4. Запись успешно добавлена в таблицу master
5. Запись успешно добавлена в таблицу master
6. Запись успешно добавлена в таблицу master
7. Запись успешно добавлена в таблицу master
8. Запись успешно добавлена в таблицу master
9. Запись успешно добавлена в таблицу master


In [137]:
postgres_to_dataframe('''SELECT * FROM master''')

,master_id,master_name,master_phone,region
0,1,Демьянов Кирилл Степанович,+7(909)291-72-77,ЗАО
1,2,Морозов Артур Тимурович,+7(999)677-11-60,ЮВАО
2,3,Ефимов Сергей Богданович,+7(916)296-55-85,СЗАО
3,4,Ушаков Артём Германович,+7(916)001-53-82,САО
4,5,Исаков Владимир Александрович,+7(925)010-47-87,СВАО
5,6,Одинцов Владислав Романович,+7(903)331-09-85,СВАО
6,7,Андреев Илья Иванович,+7(906)073-38-27,ЦАО
7,8,Черных Вадим Владимирович,+7(499)755-73-93,ЮАО
8,9,Демина Полина Валерьевна,+7(926)255-89-09,ЗелАО
9,10,Сидоров Платон Владимирович,+7(915)019-98-76,ЮЗАО


In [119]:
categories = ["Интернет", "Интернет", "Интернет", "Телевидение", "Телевидение", "Домашний телефон", "Домашний телефон", "Домашний телефон", "Домашний телефон", "Пакет услуг"]
names = ["Интернет200", "Интернет500", "Интернет1000", "ТВ-пакет 'Оптимальный'", "ТВ-пакет 'Базовый Супер'", "Повременный", "Комбинированный", "Комбинированный 400", "Безлимитный", "Интернет + ТВ"]
prices = [500, 690, 990, 120, 160, 225, 253, 449, 524, 800]
infos = ["200 мбит/с", "500 мбит/с", "1000 мбит/с", "79 каналов, 15 HD", "209 каналов, 70 HD", "гор./моб. звонки: 0 мин. | 0 мин. ; 0.6 руб./мин | 1,58 руб./мин", "гор./моб. звонки: 50 мин. | 0 мин. ; 0.59 руб./мин | 1,58 руб./мин", "гор./моб. звонки: 400 мин. | 0 мин. ; 0.58 руб./мин | 1,58 руб./мин", "гор./моб. звонки: неогр. | 0 мин. ; 0 руб./мин | 1,58 руб./мин", "500 мбит/с; 209 каналов, 70 HD"]
for i in range(0, 10):
    record_to_insert = (i+1, categories[i], names[i], prices[i], infos[i])
    insert('tariff', record_to_insert, i)

0. Запись успешно добавлена в таблицу tariff
1. Запись успешно добавлена в таблицу tariff
2. Запись успешно добавлена в таблицу tariff
3. Запись успешно добавлена в таблицу tariff
4. Запись успешно добавлена в таблицу tariff
5. Запись успешно добавлена в таблицу tariff
6. Запись успешно добавлена в таблицу tariff
7. Запись успешно добавлена в таблицу tariff
8. Запись успешно добавлена в таблицу tariff
9. Запись успешно добавлена в таблицу tariff


In [138]:
postgres_to_dataframe('''SELECT * FROM tariff''')

,tariff_id,category,tariff_name,tariff_price,info
0,1,Интернет,Интернет200,500,200 мбит/с
1,2,Интернет,Интернет500,690,500 мбит/с
2,3,Интернет,Интернет1000,990,1000 мбит/с
3,4,Телевидение,ТВ-пакет 'Оптимальный',120,"79 каналов, 15 HD"
4,5,Телевидение,ТВ-пакет 'Базовый Супер',160,"209 каналов, 70 HD"
5,6,Домашний телефон,Повременный,225,гор./моб. звонки: 0 мин. | 0 мин. ; 0.6 руб./м...
6,7,Домашний телефон,Комбинированный,253,гор./моб. звонки: 50 мин. | 0 мин. ; 0.59 руб....
7,8,Домашний телефон,Комбинированный 400,449,гор./моб. звонки: 400 мин. | 0 мин. ; 0.58 руб...
8,9,Домашний телефон,Безлимитный,524,гор./моб. звонки: неогр. | 0 мин. ; 0 руб./мин...
9,10,Пакет услуг,Интернет + ТВ,800,"500 мбит/с; 209 каналов, 70 HD"


In [121]:
names = ["Не требуется", "Интернет", "ТВ", "Домашний телефон", "Промо"]
prices = [0, 1500, 1500, 1800, 2500]

for i in range(0, 5):
    record_to_insert = (i+1, names[i], prices[i])
    insert('install', record_to_insert, i)

0. Запись успешно добавлена в таблицу install
1. Запись успешно добавлена в таблицу install
2. Запись успешно добавлена в таблицу install
3. Запись успешно добавлена в таблицу install
4. Запись успешно добавлена в таблицу install


In [139]:
postgres_to_dataframe('''SELECT * FROM install''')

,install_id,install_name,install_price
0,1,Не требуется,0
1,2,Интернет,1500
2,3,ТВ,1500
3,4,Домашний телефон,1800
4,5,Промо,2500


In [12]:
def insert_main(table_name, i):
    try:
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        columns = get_columns_names(table_name)
        columns_str = ""
        
        for i in range(0, len(columns)):
            if (i != len(columns) - 1):
                columns_str+="{},".format(columns[i])
            else:
                columns_str+=columns[i]
                
        dates = ["15.03.2022", "15.03.2022", "21.03.2022", "25.03.2022", "04.04.2022", "04.04.2022", "04.04.2022", "09.04.2022", "10.04.2022", "13.04.2022", "16.04.2022", "18.04.2022", "25.04.2022", "03.05.2022", "05.05.2022"]
        client_id = [1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        tariff_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        install_id = [1, 2, 3, 4, 5]
        agent_id = [1, 2, 3, 4, 5, 6, 7, 8]
        master_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        
        # execute the INSERT statement 
        for i in range(0, 15):
            postgres_insert_query = f""" INSERT INTO agreement ({columns_str}) VALUES
            (
            '{i+1}',
            '{dates[i]}',
            '{client_id[i]}',
            '{random.choice(tariff_id)}',
            '{random.choice(install_id)}',
            '{random.choice(agent_id)}',
            '{random.choice(master_id)}',
            '0'
            )
            """
            
            cur.execute(postgres_insert_query, i)
        
            
        conn.commit()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        print(f"Запись успешно добавлена в таблицу {table_name}")

In [127]:
insert_main('agreement', i)

Запись успешно добавлена в таблицу agreement


In [23]:
postgres_to_dataframe("""SELECT * FROM agreement ORDER BY agreement_id""")

,agreement_id,date,client_id,tariff_id,install_id,agent_id,master_id,total_cost
0,1,15.03.2022,1,1,2,6,8.0,0.0
1,2,15.03.2022,1,5,3,6,8.0,0.0
2,3,21.03.2022,2,7,4,5,10.0,0.0
3,4,25.03.2022,3,2,2,8,5.0,0.0
4,5,04.04.2022,4,7,4,7,3.0,0.0
5,6,04.04.2022,4,4,3,7,3.0,0.0
6,7,04.04.2022,4,2,2,7,3.0,0.0
7,8,09.04.2022,5,9,1,1,NaN,0.0
8,9,10.04.2022,6,8,4,3,1.0,0.0
9,10,13.04.2022,7,7,4,4,6.0,0.0


# 4. SQL Запросы: 

### Функция исполнения запросов INSERT INTO, DELETE, UPDATE и т.п.

In [6]:
def postgres_query(query):
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    try:
        cur.execute(query)
        conn.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cur.close
        return 1
    
    
    finally:
        print(f"Запрос успешно выполнен!")

## 4.1 Наглядно INSERT INTO- и DELETE-запросы:

### INSERT INTO:

In [17]:
postgres_to_dataframe("""SELECT * FROM master""").tail(5)

,master_id,master_name,master_phone,region
5,6,Одинцов Владислав Романович,+7(903)331-09-85,СВАО
6,7,Андреев Илья Иванович,+7(906)073-38-27,ЦАО
7,8,Черных Вадим Владимирович,+7(499)755-73-93,ЮАО
8,9,Демина Полина Валерьевна,+7(926)255-89-09,ЗелАО
9,10,Сидоров Платон Владимирович,+7(915)019-98-76,ЮЗАО


In [18]:
postgres_query("""INSERT INTO master VALUES (11, 'Гришаков Арсений Иванович', '+7(985)713-20-19', 'ЮЗАО')""")

Запрос успешно выполнен!


In [19]:
postgres_to_dataframe("""SELECT * FROM master""").tail(5)

,master_id,master_name,master_phone,region
6,7,Андреев Илья Иванович,+7(906)073-38-27,ЦАО
7,8,Черных Вадим Владимирович,+7(499)755-73-93,ЮАО
8,9,Демина Полина Валерьевна,+7(926)255-89-09,ЗелАО
9,10,Сидоров Платон Владимирович,+7(915)019-98-76,ЮЗАО
10,11,Гришаков Арсений Иванович,+7(985)713-20-19,ЮЗАО


### DELETE:

In [20]:
postgres_query("""DELETE FROM master WHERE master_name = 'Гришаков Арсений Иванович'""")

Запрос успешно выполнен!


In [21]:
postgres_to_dataframe("""SELECT * FROM master""").tail(5)

,master_id,master_name,master_phone,region
5,6,Одинцов Владислав Романович,+7(903)331-09-85,СВАО
6,7,Андреев Илья Иванович,+7(906)073-38-27,ЦАО
7,8,Черных Вадим Владимирович,+7(499)755-73-93,ЮАО
8,9,Демина Полина Валерьевна,+7(926)255-89-09,ЗелАО
9,10,Сидоров Платон Владимирович,+7(915)019-98-76,ЮЗАО


## 4.2 SELECT запрос с несколькими таблицами

In [45]:
main_table = '''SELECT client_name, 
                       category, 
                       tariff_name, 
                       install_name, 
                       tariff_price, 
                       install_price, 
                       personal_rate, 
                       total_cost
                                  
                        FROM agreement 
                        JOIN client ON agreement.client_id = client.client_id
                        JOIN tariff ON agreement.tariff_id = tariff.tariff_id
                        JOIN install ON agreement.install_id = install.install_id
                        ORDER BY agreement_id
                        '''

In [46]:
postgres_to_dataframe("""SELECT client_name,
                                category,
                                tariff_name,
                                install_name,
                                tariff_price,
                                install_price,
                                personal_rate,
                                total_cost
                         FROM agreement, client, tariff, install
                         WHERE agreement.client_id = client.client_id
                         AND agreement.tariff_id = tariff.tariff_id
                         AND agreement.install_id = install.install_id
                         ORDER BY agreement_id""")

,client_name,category,tariff_name,install_name,tariff_price,install_price,personal_rate,total_cost
0,Звягина Людмила Арсеньевна,Интернет,Интернет200,Интернет,500,1500,0.50,0.0
1,Звягина Людмила Арсеньевна,Телевидение,ТВ-пакет 'Базовый Супер',ТВ,160,1500,0.50,0.0
2,Базин Дмитрий Георгиевич,Домашний телефон,Комбинированный,Домашний телефон,253,1800,0.50,0.0
3,Бакшаева Арина Антоновна,Интернет,Интернет500,Интернет,690,1500,0.25,0.0
4,Мажов Филипп Денисович,Домашний телефон,Комбинированный,Домашний телефон,253,1800,0.25,0.0
5,Мажов Филипп Денисович,Телевидение,ТВ-пакет 'Оптимальный',ТВ,120,1500,0.25,0.0
6,Мажов Филипп Денисович,Интернет,Интернет500,Интернет,690,1500,0.25,0.0
7,Азарова Елена Геннадьевна,Домашний телефон,Безлимитный,Не требуется,524,0,0.25,0.0
8,Бакаринцев Игнат Ефимович,Домашний телефон,Комбинированный 400,Домашний телефон,449,1800,1.00,0.0
9,Бичурин Илья Анатольевич,Домашний телефон,Комбинированный,Домашний телефон,253,1800,0.75,0.0


## 4.3 UPDATE-Запрос

In [47]:
postgres_query("""UPDATE agreement
    SET total_cost = (SELECT ceil((tariff_price + install_price) * personal_rate)) FROM tariff, install, client
    WHERE agreement.tariff_id = tariff.tariff_id
    AND agreement.install_id = install.install_id
    AND agreement.client_id = client.client_id""")

Запрос успешно выполнен!


In [48]:
postgres_to_dataframe(main_table)

,client_name,category,tariff_name,install_name,tariff_price,install_price,personal_rate,total_cost
0,Звягина Людмила Арсеньевна,Интернет,Интернет200,Интернет,500,1500,0.50,1000.0
1,Звягина Людмила Арсеньевна,Телевидение,ТВ-пакет 'Базовый Супер',ТВ,160,1500,0.50,830.0
2,Базин Дмитрий Георгиевич,Домашний телефон,Комбинированный,Домашний телефон,253,1800,0.50,1027.0
3,Бакшаева Арина Антоновна,Интернет,Интернет500,Интернет,690,1500,0.25,548.0
4,Мажов Филипп Денисович,Домашний телефон,Комбинированный,Домашний телефон,253,1800,0.25,514.0
5,Мажов Филипп Денисович,Телевидение,ТВ-пакет 'Оптимальный',ТВ,120,1500,0.25,405.0
6,Мажов Филипп Денисович,Интернет,Интернет500,Интернет,690,1500,0.25,548.0
7,Азарова Елена Геннадьевна,Домашний телефон,Безлимитный,Не требуется,524,0,0.25,131.0
8,Бакаринцев Игнат Ефимович,Домашний телефон,Комбинированный 400,Домашний телефон,449,1800,1.00,2249.0
9,Бичурин Илья Анатольевич,Домашний телефон,Комбинированный,Домашний телефон,253,1800,0.75,1540.0


## 4.4 SELECT-запросы с вложенными SELECT, JOIN, агрегацией, сортировкой

In [88]:
postgres_to_dataframe("""SELECT category, 
                                tariff_name 
                                FROM tariff
                                
                                WHERE tariff_id IN
                        (SELECT tariff_id FROM agreement WHERE install_id IN 
                        (SELECT install_id FROM install WHERE install_name = 'Не требуется'))""")

,category,tariff_name
0,Интернет,Интернет500
1,Интернет,Интернет1000
2,Домашний телефон,Безлимитный


In [27]:
postgres_to_dataframe("""SELECT client_name, 
                                count(total_cost) as service_count, 
                                sum(total_cost) 
                                FROM client, agreement
                                
                         WHERE agreement.client_id = client.client_id
                         GROUP BY client.client_id
                         HAVING sum(total_cost) > avg(total_cost)
                         ORDER BY client_name""") 

,client_name,service_count,sum
0,Звягина Людмила Арсеньевна,2,1830.0
1,Мажов Филипп Денисович,3,1467.0


In [26]:
postgres_to_dataframe("""SELECT date, 
                                master_name, 
                                region, 
                                agent_name, 
                                personnel_num as agent_num 
                                FROM agreement
                                
                         JOIN master ON agreement.master_id = master.master_id
                         JOIN agent ON agreement.agent_id = agent.agent_id
                         
                         WHERE tariff_id IN (SELECT tariff_id FROM tariff WHERE category = 'Домашний телефон')
                         ORDER BY date""") 

,date,master_name,region,agent_name,agent_num
0,04.04.2022,Ефимов Сергей Богданович,СЗАО,Ерофеев Богдан Ярославович,33
1,10.04.2022,Демьянов Кирилл Степанович,ЗАО,Устинова Вероника Михайловна,65
2,13.04.2022,Одинцов Владислав Романович,СВАО,Моисеев Александр Ильич,82
3,16.04.2022,Морозов Артур Тимурович,ЮВАО,Смирнова Виктория Константиновна,66
4,21.03.2022,Сидоров Платон Владимирович,ЮЗАО,Баранов Иван Иванович,84
5,25.04.2022,Сидоров Платон Владимирович,ЮЗАО,Моисеев Александр Ильич,82


In [108]:
postgres_to_dataframe("""SELECT client_name, 
                                category, 
                                tariff_name, 
                                total_cost 
                                FROM agreement
                                
                         JOIN client ON agreement.client_id = client.client_id
                         JOIN tariff ON agreement.tariff_id = tariff.tariff_id
                         
                         WHERE agreement.client_id IN 
                         
                         (SELECT client_id FROM agreement GROUP BY agreement.client_id
                         HAVING sum(total_cost) > avg(total_cost))
                         ORDER BY client_name""") 

,client_name,category,tariff_name,total_cost
0,Звягина Людмила Арсеньевна,Интернет,Интернет200,1000.0
1,Звягина Людмила Арсеньевна,Телевидение,ТВ-пакет 'Базовый Супер',830.0
2,Мажов Филипп Денисович,Интернет,Интернет500,548.0
3,Мажов Филипп Денисович,Телевидение,ТВ-пакет 'Оптимальный',405.0
4,Мажов Филипп Денисович,Домашний телефон,Комбинированный,514.0
